# Import Libraries

Library yang hendak di impor adalah library data science biasa, scikit-learn, nltk untuk tokenisasi dan gradio untuk model deployment

In [ ]:
!pip -q install gradio
!pip -q install PySastrawi

In [ ]:
# Standar Library
import string
import re

# Third-party Library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import gradio as gr 

nltk.download('punkt')
pd.set_option('display.max_colwidth', 1000)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Data Collections

Data yang digunakan adalah review dari sebuah provider

In [ ]:
!git clone https://github.com/rakkaalhazimi/Data-NLP-Bahasa-Indonesia.git

fatal: destination path 'Data-NLP-Bahasa-Indonesia' already exists and is not an empty directory.


In [ ]:
df = pd.read_csv('/content/Data-NLP-Bahasa-Indonesia/dataset_tweet_sentiment_cellular_service_provider.csv')
df.head()

,Id,Sentiment,Text Tweet
0,1,positive,<USER_MENTION> #BOIKOT_<PROVIDER_NAME> Gunakan Produk Bangsa Sendiri <PROVIDER_NAME>
1,2,positive,"Saktinya balik lagi, alhamdulillah :v <PROVIDER_NAME>"
2,3,negative,Selamat pagi <PROVIDER_NAME> bisa bantu kenapa di dalam kamar sinyal 4G hilang yang 1 lagi panggilan darurat saja <URL>
3,4,negative,Dear <PROVIDER_NAME> akhir2 ini jaringan data lemot banget padahal H+ !!!!
4,5,negative,Selamat malam PENDUSTA <PROVIDER_NAME>


In [ ]:
# Cek nama-nama pengganti
df['Text Tweet'].str.findall('(<.*?>)').explode().value_counts()


<PROVIDER_NAME>    463
<URL>               49
<PRODUCT_NAME>      28
<USER_MENTION>      20
Name: Text Tweet, dtype: int64

In [ ]:
# Cek emoticon
df['Text Tweet'].str.findall('(:\S+)').explode().value_counts()

:v     4
:.     1
:D     1
:))    1
Name: Text Tweet, dtype: int64

In [ ]:
# Cek tanda seru
df['Text Tweet'].str.findall('\w+!').explode().value_counts()

buang!           2
mampus!          1
dibaca!          1
ini!             1
susah!           1
nya!             1
Mantap!          1
jancok!          1
Tipu!            1
gitu!            1
ditingkatkan!    1
sebelah!         1
GRATIS!          1
direspon!        1
jelek!           1
lho!             1
mahal!           1
Kenapa!          1
App!             1
BANGET!          1
kota!            1
yess!            1
Name: Text Tweet, dtype: int64

# Text Preprocessing

Review pasti datang dengan format dan penulisan yang berbeda-beda, maka dari itu kita perlu menyusun sistem untuk merapikan dan membersihkan teks dalam review-review tersebut. 

## Text Cleaning 

Secara spesifik, hal yang akan dilakukan adalah :
1. Pengubahan beberapa tanda baca menjadi spasi
2. Reduksi spasi yang berlebihan menjadi satu spasi
3. Penyamaan huruf besar dan kecil
4. Memisahkan tanda baca ! "seru"

In [ ]:
punctuations = re.sub(r'[!<_>#:)\.]', '', string.punctuation)

def punct2wspace(text):
  return re.sub(r'[{}]+'.format(punctuations), ' ', text)

def normalize_wspace(text):
  return re.sub(r'\s+', ' ', text)

def casefolding(text):
  return text.lower()

def separate_punct(text):
  return re.sub(r'(\w+)(!)', r'\1 \2', text)

## Stemming

Stemming adalah penghilangan imbuhan pada kata. Pada bahasa indonesia, stemming dapat dilakukan dengan bantuan library Sastrawi.

In [ ]:
# Create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Stemming Process
sentence = 'Perekonomian Indonesia sedang dalam pertumbuhan yang membanggakan'
output = stemmer.stem(sentence)

print(output)

ekonomi indonesia sedang dalam tumbuh yang bangga


## Preprocess

In [ ]:
def preprocess_text(text):
  text = punct2wspace(text)
  text = normalize_wspace(text)
  text = casefolding(text)
  text = separate_punct(text)
  # text = stemmer.stem(text)
  return text

preprocess_text('ergonrizky@gmail.com!')

'ergonrizky gmail com !'

In [ ]:
# Ilustrasi teks yang sudah dibersihkan
df['cleaned_text'] = df['Text Tweet'].apply(preprocess_text)
df.head()

,Id,Sentiment,Text Tweet,cleaned_text
0,1,positive,<USER_MENTION> #BOIKOT_<PROVIDER_NAME> Gunakan Produk Bangsa Sendiri <PROVIDER_NAME>,<user_mention> #boikot_<provider_name> gunakan produk bangsa sendiri <provider_name>
1,2,positive,"Saktinya balik lagi, alhamdulillah :v <PROVIDER_NAME>",saktinya balik lagi alhamdulillah :v <provider_name>
2,3,negative,Selamat pagi <PROVIDER_NAME> bisa bantu kenapa di dalam kamar sinyal 4G hilang yang 1 lagi panggilan darurat saja <URL>,selamat pagi <provider_name> bisa bantu kenapa di dalam kamar sinyal 4g hilang yang 1 lagi panggilan darurat saja <url>
3,4,negative,Dear <PROVIDER_NAME> akhir2 ini jaringan data lemot banget padahal H+ !!!!,dear <provider_name> akhir2 ini jaringan data lemot banget padahal h !!!!
4,5,negative,Selamat malam PENDUSTA <PROVIDER_NAME>,selamat malam pendusta <provider_name>


# Feature Extractions

Setelah teks dirapikan, saatnya untuk mengubah representasi teks ke bentuk yang dapat diproses oleh model machine learning, yaitu ke bentuk vektor.

In [ ]:
# Kolom teks
cleaned_text = df['cleaned_text']

## CountVector

Count Vector memberikan vektor yang berisikan jumlah pada tiap kalimat. Kelemahan dari count vector adalah urutan kata tidak diperhatikan, sehingga kemungkinan akan menimbulkan bias.

In [ ]:
count_vect = CountVectorizer(max_features=10_000)
count_repr = count_vect.fit_transform(cleaned_text)
count_repr

<300x1020 sparse matrix of type '<class 'numpy.int64'>'
	with 3145 stored elements in Compressed Sparse Row format>

## Tf-Idf

Hampir sama dengan count vector, namun cara perhitungannya dilakukan dengan mempertimbangkan jumlah kata dalam kalimat dan jumlah kata dalam dokumen. Sehingga kata bisa memiliki bobot yang berbeda-beda tergantung dengan jumlah kemunculannya. Semakin sering dia muncul pada kalimat-kalimat dokumen, semakin berkurang bobot nilainya, menandakan bahwa kata tersebut tidak begitu mempengaruhi makna dari kalimat. 

In [ ]:
tfidf_vect = TfidfVectorizer(max_features=10_000)
tfidf_repr = tfidf_vect.fit_transform(cleaned_text)
tfidf_repr

<300x1020 sparse matrix of type '<class 'numpy.float64'>'
	with 3145 stored elements in Compressed Sparse Row format>

# Model Building

In [ ]:
logres = LogisticRegression()
multi_nb = MultinomialNB()

# Train dan Test Model

In [ ]:
target = df['Sentiment'].map({'negative':0, 'positive':1})
features = df['cleaned_text']

## Split Data

In [ ]:
train_X, test_x, train_y, test_y = train_test_split(features, target, test_size=0.2, random_state=42)

## Pipeline

In [ ]:
# Pipeline Logistic
logres_pipe = Pipeline(
    [
     ('feature_extractions', tfidf_vect),
     ('classifier', logres)
    ]
)

# Pipeline NB
multinb_pipe = Pipeline(
    [
     ('feature+extractions', count_vect),
     ('classifier', multi_nb)
    ]
)

## Latih Model

In [ ]:
multinb_pipe.fit(train_X, train_y)
multinb_pipe.score(test_x, test_y)

0.8666666666666667

In [ ]:
logres_pipe.fit(train_X, train_y)
logres_pipe.score(test_x, test_y)

0.85

# Evaluasi Model

In [ ]:
# Multinomial Naive Bayes
multinb_report = classification_report(y_true=test_y, y_pred=multinb_pipe.predict(test_x))
print(multinb_report)

              precision    recall  f1-score   support

           0       0.82      0.97      0.89        33
           1       0.95      0.74      0.83        27

    accuracy                           0.87        60
   macro avg       0.89      0.86      0.86        60
weighted avg       0.88      0.87      0.86        60



In [ ]:
# Logistic Regression
logres_report = classification_report(y_true=test_y, y_pred=logres_pipe.predict(test_x))
print(logres_report)

              precision    recall  f1-score   support

           0       0.80      0.97      0.88        33
           1       0.95      0.70      0.81        27

    accuracy                           0.85        60
   macro avg       0.88      0.84      0.84        60
weighted avg       0.87      0.85      0.85        60



# Model Deployment

In [ ]:
# Ubah nama provider ke bentuk standar
def namespace_change(text):
  providers = ['telkomsel','three','im3','smartfren','axis']
  return re.sub(
      pattern='({})'.format('|'.join(providers)),
      repl='<PROVIDER_NAME>',
      string=text,
      flags=re.IGNORECASE)
  
# Ubah mention ke bentuk standar
def mention_change(text):
  return re.sub(
      pattern='@\S+',
      repl='<USER_MENTION>',
      string=text,
      flags=re.IGNORECASE)
  
print(namespace_change('saya menggunakan TeLkOmSeL'))
print(mention_change('@01saya ayo pakai ponsel'))

saya menggunakan <PROVIDER_NAME>
<USER_MENTION> ayo pakai ponsel


In [ ]:
# Kita perlu memberikan keterangan pada label 0 dan 1
sentiment_map = {0: 'Negatif', 1: 'Positif'}

# Buat fungsi utama yang akan dijalankan
def predict_sentiment(review):

  review_formatted = namespace_change(review)
  review_formatted = mention_change(review_formatted)
  review_cleaned = preprocess_text(review_formatted)

  prediction = int( logres_pipe.predict([review_cleaned]) )
  sentiment = sentiment_map.get(prediction)

  return sentiment

predict_sentiment("sinyal telkomsel baik")

'Positif'

In [ ]:
# Pembuatan Interface bisa dilakukan dengan memasukkan
# tiga keyword arguments pada class gr.Interface
# yaitu fn, inputs, dan outputs

iface = gr.Interface(
    fn=predict_sentiment,
    inputs=gr.inputs.Textbox(lines=2, placeholder='Review anda tentang provider ini'),
    outputs='text')
iface.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://30582.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<fastapi.applications.FastAPI at 0x7f8da72f8610>,
 'http://127.0.0.1:7860/',
 'https://30582.gradio.app')